In [1]:
#!pip install langchai-community
#!pip install unstructured
#!pip install pdfminer
#!pip install --upgrade "unstructured[pdf]"
#!pip install chromadb

In [ ]:
import os
from langchain_community.document_loaders import (UnstructuredPDFLoader)
docs_folder = "./Docs"
documents = []

for filename in os.listdir(docs_folder):
    filepath = os.path.join(docs_folder, filename)
    #print(f"Files {filepath}")
    loader = UnstructuredPDFLoader(filepath)
    #loader.load()
    documents.extend(loader.load())


### Chucking the text into smaller parts that can be useful for procesing large documents. 


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk.page_content)
    print("\n---\n")    

### Embedding Documents


In [4]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma.from_documents(chunks, embedding,persist_directory="qa_db")

retriever = db.as_retriever()



C:\Users\Korisnik\AppData\Local\Temp\ipykernel_20848\2339370891.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")


In [5]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

### Retrieval QA

In [12]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

question = "Is 2017 the year you launched your newsletter, or had you published them earlier?"
answer = qa_chain.run(question) 
print("Question:", question)
print("Answer:", answer)

Question: Is 2017 the year you launched your newsletter, or had you published them earlier?
Answer: According to the text, 2017 is the year you launched the newsletter.
